In [1]:
!pip install -U "transformers==4.40.0" --upgrade

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `llama2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama2`


In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    task = "text-generation",
    model = model_id,
    model_kwargs = {"torch_dtype": torch.bfloat16},
    device = 'cuda',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
messages = [
    {"role": "system", "content": " You can classify text sentiment as bullish, bearish, or neutral!"},
    {"role": "user", "content": "TSLA slashes model Y prices?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)


print(prompt)
prompt_full_explanation = prompt

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You can classify text sentiment as bullish, bearish, or neutral!<|eot_id|><|start_header_id|>user<|end_header_id|>

TSLA slashes model Y prices?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [43]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


A juicy one!

Here's a sentiment analysis:

**Bullish**

The fact that TSLA (Tesla) is slashing Model Y prices could be seen as a sign that the company is trying to boost sales and clear inventory. This could be a strategic move to increase market share and stay competitive in the electric vehicle market. Additionally, a lower price point could make the Model Y more attractive to potential customers, which could lead to increased demand and revenue for the company.

**Bearish**

On the other hand, the price cut could also be seen as a sign of weakness in the demand for the Model Y. If the company is having trouble selling the cars at the previous price point, it may indicate that there is less demand for the vehicle than initially thought. This could lead to concerns about the company's ability to meet its sales targets and potentially impact the stock price.

**Neutral**

Another way to look at it is that the price cut is simply a normal part of the business cycle. Companies often adj

In [6]:
messages = [
    {"role": "system", "content": " You can classify text sentiment as bullish, bearish, or neutral! Respond with only one word"},
    {"role": "user", "content": "TSLA slashes model Y prices?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You can classify text sentiment as bullish, bearish, or neutral! Respond with only one word<|eot_id|><|start_header_id|>user<|end_header_id|>

TSLA slashes model Y prices?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [44]:
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


A juicy piece of news!

Let's analyze the sentiment:

**TSLA slashes Model Y prices**

This headline has a bearish sentiment. The word "slashes" implies a significant reduction in price, which is likely to have a negative impact on the company's revenue and profitability. The use of the word "slashes" also adds a sense of urgency and suddenness to the event, which can contribute to a bearish tone.

The sentiment can be classified as:

* Bearish: 8/10


# Load the dataset

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('/content/drive/MyDrive/tla_sentiment.csv')

In [12]:
df

,time_published,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
0,20241024T140300,ETFs to Tap Tesla's Solid Q3 Earnings,https://www.zacks.com/stock/news/2356847/etfs-...,['Sweta Killa'],Tesla reports its biggest quarterly profit in ...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.284626,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.6764...",0.676457,0.431176,1,Bullish,Bullish
1,20241024T150857,Why Tesla Stock Is Soaring Today,https://www.fool.com/investing/2024/10/24/why-...,['Howard Smith'],Tesla stock shot higher today after its third-...,https://g.foolcdn.com/editorial/images/795190/...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.309466,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8779...",0.877990,0.510945,1,Bullish,Bullish
2,20241024T155500,3 Reasons to Buy Tesla Stock Like There's No T...,https://www.fool.com/investing/2024/10/24/3-re...,['Rick Munarriz'],"Investors are liking Tesla's latest report, bu...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.226688,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8088...",0.808852,0.332660,1,Somewhat_Bullish,Bullish
3,20241025T073809,Elon Musk Sees $33.5B Wealth Surge In A Day As...,https://www.benzinga.com/markets/equities/24/1...,['Pooja Rajkumari'],Elon Musk experienced a dramatic increase in h...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.449339,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.4950...",0.495015,0.445558,1,Bullish,Bullish
4,20241025T094250,Tesla Shares Slip In Pre-Market A Day After Ma...,https://www.benzinga.com/markets/equities/24/1...,['Pooja Rajkumari'],Shares of Tesla Inc. TSLA experienced a declin...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.352444,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.2712...",0.271274,0.124063,1,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,20241115T160148,Dawn Project testing of Tesla Full Self-Drivin...,https://www.benzinga.com/pressreleases/24/11/g...,['Globe Newswire'],Tesla fan confirms safety advocacy group's tes...,https://ml.globenewswire.com/Resource/Download...,Benzinga,General,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.037264,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8991...",0.899125,0.047364,1,Neutral,Neutral
72,20241116T133000,"Losing the $7,500 Electric Vehicle Credit Coul...",https://www.fool.com/investing/2024/11/16/how-...,['Travis Hoium'],President-elect Trump wants to eliminate the $...,https://g.foolcdn.com/editorial/images/798036/...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.294126,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.9591...",0.959190,0.598081,1,Bullish,Bullish
73,20241116T233000,Is Tesla Still a Millionaire-Maker Stock?,https://www.fool.com/investing/2024/11/16/is-t...,['Will Ebiefung'],Shares in this electric vehicle leader have so...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.4...",0.334518,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5665...",0.566543,0.452913,1,Bullish,Bullish
74,20241117T062115,Elon Musk Reacts After Brazilian President's W...,https://www.benzinga.com/24/11/42025443/brazil...,['Anan Ashraf'],Brazil's first Lady Rosangela da Silva reporte...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,G

In [13]:
df['time_published'] = pd.to_datetime(df['time_published'])



In [14]:
df

,time_published,title,url,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker_relevance_TSLA,ticker_sentiment_TSLA,num_tickers,detailed_original_label,label
0,2024-10-24 14:03:00,ETFs to Tap Tesla's Solid Q3 Earnings,https://www.zacks.com/stock/news/2356847/etfs-...,['Sweta Killa'],Tesla reports its biggest quarterly profit in ...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,NaN,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.284626,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.6764...",0.676457,0.431176,1,Bullish,Bullish
1,2024-10-24 15:08:57,Why Tesla Stock Is Soaring Today,https://www.fool.com/investing/2024/10/24/why-...,['Howard Smith'],Tesla stock shot higher today after its third-...,https://g.foolcdn.com/editorial/images/795190/...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.309466,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8779...",0.877990,0.510945,1,Bullish,Bullish
2,2024-10-24 15:55:00,3 Reasons to Buy Tesla Stock Like There's No T...,https://www.fool.com/investing/2024/10/24/3-re...,['Rick Munarriz'],"Investors are liking Tesla's latest report, bu...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.226688,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.8088...",0.808852,0.332660,1,Somewhat_Bullish,Bullish
3,2024-10-25 07:38:09,Elon Musk Sees $33.5B Wealth Surge In A Day As...,https://www.benzinga.com/markets/equities/24/1...,['Pooja Rajkumari'],Elon Musk experienced a dramatic increase in h...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.449339,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.4950...",0.495015,0.445558,1,Bullish,Bullish
4,2024-10-25 09:42:50,Tesla Shares Slip In Pre-Market A Day After Ma...,https://www.benzinga.com/markets/equities/24/1...,['Pooja Rajkumari'],Shares of Tesla Inc. TSLA experienced a declin...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.3...",0.352444,Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.2712...",0.271274,0.124063,1,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2024-11-15 16:01:48,Dawn Project testing of Tesla Full Self-Drivin...,https://www.benzinga.com/pressreleases/24/11/g...,['Globe Newswire'],Tesla fan confirms safety advocacy group's tes...,https://ml.globenewswire.com/Resource/Download...,Benzinga,General,www.benzinga.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.037264,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.8991...",0.899125,0.047364,1,Neutral,Neutral
72,2024-11-16 13:30:00,"Losing the $7,500 Electric Vehicle Credit Coul...",https://www.fool.com/investing/2024/11/16/how-...,['Travis Hoium'],President-elect Trump wants to eliminate the $...,https://g.foolcdn.com/editorial/images/798036/...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Manufacturing', 'relevance_score':...",0.294126,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.9591...",0.959190,0.598081,1,Bullish,Bullish
73,2024-11-16 23:30:00,Is Tesla Still a Millionaire-Maker Stock?,https://www.fool.com/investing/2024/11/16/is-t...,['Will Ebiefung'],Shares in this electric vehicle leader have so...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.4...",0.334518,Somewhat-Bullish,"[{'ticker': 'TSLA', 'relevance_score': '0.5665...",0.566543,0.452913,1,Bullish,Bullish
74,2024-11-17 06:21:15,Elon Musk Reacts After Brazilian President's W...,https://www.benzinga.com/24/11/42025443/brazil...,['Anan Ashraf'],Brazil's first Lady Rosangela da Silva reporte...,https://cdn.benzinga.com

In [23]:
df = df[['time_published','summary','label']]

In [24]:
def get_prompt(summary):
    messages = [
      {"role": "system", "content": " You can classify text sentiment as bullish, bearish, or neutral! Respond with only one word"},
      {"role": "user", "content": summary},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
    )
    return prompt

def get_prompt_full_explanation(summary):
    messages = [
      {"role": "system", "content": " You can classify text sentiment as bullish, bearish, or neutral!"},
      {"role": "user", "content": summary},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
    )
    return prompt

In [25]:
df['prompt']=df.apply(lambda l:get_prompt(l['summary']),axis=1)
df['prompt_full_explanation']=df.apply(lambda l:get_prompt_full_explanation(l['summary']),axis=1)

<ipython-input-25-cf9d99f5b539>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompt']=df.apply(lambda l:get_prompt(l['summary']),axis=1)
<ipython-input-25-cf9d99f5b539>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompt_full_explanation']=df.apply(lambda l:get_prompt_full_explanation(l['summary']),axis=1)


In [26]:
print(df['prompt'].iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You can classify text sentiment as bullish, bearish, or neutral! Respond with only one word<|eot_id|><|start_header_id|>user<|end_header_id|>

Tesla reports its biggest quarterly profit in more than a year and issues upbeat forecasts for 2025.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [29]:
print(df['prompt'].iloc[0])
print(df['label'].iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You can classify text sentiment as bullish, bearish, or neutral! Respond with only one word<|eot_id|><|start_header_id|>user<|end_header_id|>

Tesla reports its biggest quarterly profit in more than a year and issues upbeat forecasts for 2025.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Bullish


In [30]:
outputs = pipeline(
    df.prompt.iloc[0],
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    temperature=0,
    #top_p=0.9,
)

outputs[0]["generated_text"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou can classify text sentiment as bullish, bearish, or neutral! Respond with only one word<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTesla reports its biggest quarterly profit in more than a year and issues upbeat forecasts for 2025.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBullish'

In [31]:
outputs[0]['generated_text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou can classify text sentiment as bullish, bearish, or neutral! Respond with only one word<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTesla reports its biggest quarterly profit in more than a year and issues upbeat forecasts for 2025.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBullish'

In [32]:
outputs[0]["generated_text"].split("<|end_header_id|>\n\n")[-1]

'Bullish'

Split into train/val/test for later comparison for when we fine tune.
For simplicity we split based on time.
First 60% train
Next 20% val
Next 20% test

In [33]:
train_end_point = int(df.shape[0]*0.6)
val_end_point = int(df.shape[0]*0.8)
df_train = df.iloc[:train_end_point,:]
df_val = df.iloc[train_end_point:val_end_point,:]
df_test = df.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)

(45, 5) (16, 5) (15, 5)


In [34]:
test_outputs = pipeline(df_test['prompt'].tolist(),   max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,

)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_toke

In [35]:
 test_predictions=[output[0]['generated_text'].split("<|end_header_id|>\n\n")[-1] for output in test_outputs]
 test_predictions[0:10]

['Bullish',
 'Bullish',
 'Neutral',
 'Neutral',
 'Bullish',
 'Neutral',
 'Bearish',
 'Bullish',
 'Bullish',
 'Bearish']

In [36]:
df_test['predictions'] =  test_predictions

<ipython-input-36-1c3124cc687f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions'] =  test_predictions


In [39]:
df_test.predictions[df_test['predictions'].isin(['Bearish','Neutral','Bullish'])].value_counts()

,count
predictions,
Bullish,7
Bearish,5
Neutral,4


In [40]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
y_test = df_test.label
y_pred = df_test.predictions

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[0 0 0]
 [2 7 1]
 [3 0 3]]

Classification Report:
              precision    recall  f1-score   support

     Bearish       0.00      0.00      0.00         0
     Bullish       1.00      0.70      0.82        10
     Neutral       0.75      0.50      0.60         6

    accuracy                           0.62        16
   macro avg       0.58      0.40      0.47        16
weighted avg       0.91      0.62      0.74        16

Balanced Accuracy Score: 0.6
Accuracy Score: 0.625


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

In [41]:
df_test

,time_published,summary,label,prompt,prompt_full_explanation,predictions
60,2024-11-13 06:02:12,Tesla Inc.'s TSLA gigafactory in Nevada built ...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bullish
61,2024-11-13 23:01:28,Electric vehicle giant Tesla Inc TSLA has ofte...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bullish
62,2024-11-14 08:25:48,"Tesla Inc TSLA is not prioritizing the ""afflue...",Neutral,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Neutral
63,2024-11-14 12:23:42,President-elect Donald Trump's naming of Tesla...,Neutral,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Neutral
64,2024-11-14 19:16:58,Tesla Inc TSLA has soared since Donald Trump w...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bullish
65,2024-11-14 20:54:45,Tesla Inc TSLA shares are moving lower Thursda...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Neutral
66,2024-11-15 03:52:55,Prominent Tesla Inc TSLA investor Ross Gerber ...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bearish
67,2024-11-15 09:04:10,Tesla Inc TSLA is offering three months of fre...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bullish
68,2024-11-15 11:15:00,Elon Musk made a big bet on Donald Trump this ...,Bullish,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bullish
69,2024-11-15 12:08:46,Tesla Inc TSLA CEO Elon Musk on Thursday calle...,Neutral,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,Bearish
